In [1]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random
random.seed(1)
np.random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)
import time
start_time = time.time()

In [2]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:

%cd /content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/3_Classification

%ls

/content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Scare Quotes/3_Classification
annotation_Drew_sample_scare_quotes.csv  scare_quote_feature_contributions_plot.png
drew_gold_standard_scare_quote_500.csv   scare_quote_feature_importance_plot.png
gold_standard_scare_quotes_100.csv       scare_quote_rf_svm_classifiers.ipynb
gold_standard_scare_quotes_500.csv       scare_quotes_accuracy.gsheet
predictions.csv                          scare_quotes_model_results.csv
scare_quote_classification.ipynb         summary_models_scare_quote_bag_of_words.csv


In [4]:
# Read the Annie Sample csv file
df1 = pd.read_csv("gold_standard_scare_quotes_500.csv") #Annie Sample

# Read the Drew Sample csv file
df2 = pd.read_csv("drew_gold_standard_scare_quote_500.csv") #Drew Sample
# Bind the rows of the two dataframes
df = pd.concat([df1, df2], ignore_index=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sentence_id  1003 non-null   int64  
 1   sentence     995 non-null    object 
 2   quoted_text  994 non-null    object 
 3   scare_label  995 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.5+ KB


In [5]:

df['label'] = df['scare_label']
df.drop(['sentence_id','scare_label'], axis=1)
del df['sentence_id'], df['scare_label']
# Drop rows with any NaN values
df.dropna(inplace=True)
df.head(3)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994 entries, 0 to 1002
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sentence     994 non-null    object 
 1   quoted_text  994 non-null    object 
 2   label        994 non-null    float64
dtypes: float64(1), object(2)
memory usage: 31.1+ KB


In [6]:
cols = ['sentence', 'quoted_text']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df.head(3)

,sentence,quoted_text,label,text
0,"Pt stated year as ""[**2159**]"" and when asked ...",[**2159**],0.0,"Pt stated year as ""[**2159**]"" and when asked ..."
1,Speech evaluation recommended behavioral modif...,break,0.0,Speech evaluation recommended behavioral modif...
2,"psychosocial: pt tearful several times, mouthi...",go home,0.0,"psychosocial: pt tearful several times, mouthi..."


In [7]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [8]:
class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        #print(self.targets.shape)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [9]:
newdf = df[['text','label']]
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994 entries, 0 to 1002
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    994 non-null    object 
 1   label   994 non-null    float64
dtypes: float64(1), object(1)
memory usage: 23.3+ KB


In [10]:
train_size = 0.8
train_data=newdf.sample(frac=train_size,random_state=0)
test_data=newdf.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(newdf.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (994, 2)
TRAIN Dataset: (795, 2)
TEST Dataset: (199, 2)


In [11]:
train_data['label'].unique()

array([0., 1.])

In [12]:
test_data['label'].unique()

array([0., 1.])

In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [14]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2) #output labels = 1 for binary classification

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [15]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [16]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [17]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [18]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [19]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss: 0.7395997643470764
Training Accuracy: 12.5


100it [01:10,  1.42it/s]


The Total Accuracy for Epoch 0: 76.85534591194968
Training Loss Epoch: 0.5390162934362889
Training Accuracy Epoch: 76.85534591194968


1it [00:00,  3.29it/s]

...

The Total Accuracy for Epoch 8: 99.49685534591195
Training Loss Epoch: 0.019048852005507797
Training Accuracy Epoch: 99.49685534591195


1it [00:00,  2.07it/s]

Training Loss: 0.003140160581097007
Training Accuracy: 100.0


100it [01:14,  1.35it/s]

The Total Accuracy for Epoch 9: 98.99371069182389
Training Loss Epoch: 0.03286175006069243
Training Accuracy Epoch: 98.99371069182389


In [20]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [21]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss=0
    nb_tr_steps=0
    nb_tr_examples=0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu


Initial Run -- one interation

In [22]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")



1it [00:00,  2.58it/s]

Validation Loss: 1.823885202407837
Validation Accuracy: 62.5


25it [00:06,  3.99it/s]


Validation Loss Epoch: 1.0064591727685184
Validation Accuracy Epoch: 78.39195979899498
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       156
           1       0.50      0.28      0.36        43

    accuracy                           0.78       199
   macro avg       0.66      0.60      0.61       199
weighted avg       0.75      0.78      0.76       199

Confusion Matrix:
[[144  12]
 [ 31  12]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(853), tensor(21), tensor(12...         1          1
1  [tensor(0), tensor(31371), tensor(257), tensor...         0          0
2  [tensor(0), tensor(15561), tensor(118), tensor...         0          0
3  [tensor(0), tensor(4148), tensor(5237), tensor...         0          0
4  [tensor(0), tensor(1708), tensor(172), tensor(...         0          0


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  4.16it/s]

Validation Loss: 0.14393769204616547
Validation Accuracy: 87.5


25it [00:06,  4.10it/s]


Validation Loss Epoch: 1.006149370293133
Validation Accuracy Epoch: 78.39195979899498
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       156
           1       0.50      0.28      0.36        43

    accuracy                           0.78       199
   macro avg       0.66      0.60      0.61       199
weighted avg       0.75      0.78      0.76       199

Confusion Matrix:
[[144  12]
 [ 31  12]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(47217), tensor(3850), tenso...         0          0
1  [tensor(0), tensor(487), tensor(21824), tensor...         0          0
2  [tensor(0), tensor(104), tensor(35), tensor(22...         0          0
3  [tensor(0), tensor(1121), tensor(1285), tensor...         0          0
4  [tensor(0), tensor(12), tensor(6442), tensor(3...         0          0
test accuracy = 78.39%
Elapsed Time: 827.74 seconds


In order to account for variations in GPU states, we ran each of the models 10 times and provide the mean accuracy


## Predictions

In [ ]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)
        token_type_ids = torch.tensor([inputs["token_type_ids"]], dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)

        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return df


In [ ]:
predicted_classes = predict_on_dataframe(test_data, model, tokenizer, max_len=128)
print(predicted_classes)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


                                                  text  label  predictions
0    Doing well at 11 [**Hospital Ward Name 890**] ...    0.0            0
1    ALLERGIES: Hydralazine causes "pericardial rub...    1.0            0
2    As per above complained of chest pain "below t...    0.0            0
3    The patient states that he was in his usual st...    0.0            0
4    # Code: FULL Revisited during ICU course, Dr. ...    0.0            0
..                                                 ...    ...          ...
194   EGD revealed "watermelon stomach" and patient...    0.0            0
195  NPN 7A-7P EVENTS: pt extubated, code status ch...    0.0            0
196  CCU NPN 1900-0700 S: " I'm not going to MRI- I...    1.0            0
197  Interpreter only able to understand pt say "I'...    0.0            0
198  pt c/o "sour stomach" x 1- tx with reglan with...    0.0            0

[199 rows x 3 columns]


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

# Assuming predicted_classes is your dataframe
true_labels = predicted_classes['label'].tolist()
predicted_labels = predicted_classes['predictions'].tolist()

def bootstrap_metric(y_true, y_pred, metric_func, n_iterations=1000):
    bootstrap_values = []
    n_samples = len(y_true)
    for _ in range(n_iterations):
        indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        bootstrap_true = np.array(y_true)[indices]
        bootstrap_pred = np.array(y_pred)[indices]
        value = metric_func(bootstrap_true, bootstrap_pred)
        bootstrap_values.append(value)
    return bootstrap_values

# Bootstrapping each metric
bootstrap_accuracy = bootstrap_metric(true_labels, predicted_labels, accuracy_score)
bootstrap_f1_pos = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: f1_score(y_true, y_pred, pos_label=1))
bootstrap_precision_pos = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: precision_score(y_true, y_pred, pos_label=1))
bootstrap_recall_pos = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: recall_score(y_true, y_pred, pos_label=1))
bootstrap_f1_macro = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'))
bootstrap_precision_macro = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: precision_score(y_true, y_pred, average='macro'))
bootstrap_recall_macro = bootstrap_metric(true_labels, predicted_labels, lambda y_true, y_pred: recall_score(y_true, y_pred, average='macro'))

# Create dataframe with each iteration's metric values
df_bootstrap = pd.DataFrame({
    'Accuracy': bootstrap_accuracy,
    'F1_Positive': bootstrap_f1_pos,
    'Precision_Positive': bootstrap_precision_pos,
    'Recall_Positive': bootstrap_recall_pos,
    'F1_Macro': bootstrap_f1_macro,
    'Precision_Macro': bootstrap_precision_macro,
    'Recall_Macro': bootstrap_recall_macro
})

# Calculate the mean and 95% confidence intervals for each metric
confidence_level = 0.95
lower_bound = (1.0 - confidence_level) / 2.0 * 100
upper_bound = (confidence_level + ((1.0 - confidence_level) / 2.0)) * 100

metrics_summary = {
    'Metric': [],
    'Mean': [],
    'Lower 95% CI': [],
    'Upper 95% CI': []
}

for col in df_bootstrap.columns:
    metrics_summary['Metric'].append(col)
    metrics_summary['Mean'].append(np.mean(df_bootstrap[col]))
    metrics_summary['Lower 95% CI'].append(np.percentile(df_bootstrap[col], lower_bound))
    metrics_summary['Upper 95% CI'].append(np.percentile(df_bootstrap[col], upper_bound))

df_summary = pd.DataFrame(metrics_summary)

print(df_summary)


               Metric      Mean  Lower 95% CI  Upper 95% CI
0            Accuracy  0.753528      0.693467      0.814070
1         F1_Positive  0.345484      0.202889      0.477643
2  Precision_Positive  0.407556      0.239921      0.578990
3     Recall_Positive  0.303448      0.170124      0.450025
4            F1_Macro  0.597586      0.515906      0.677486
5     Precision_Macro  0.614217      0.522480      0.703409
6        Recall_Macro  0.592654      0.522648      0.668048


Notes for next time-- want to turn this script into a function where I can pass through the hyperparameters.

In [ ]:
# svm baseline

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.svm import SVC

#from sklearn.pipeline import Pipeline
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto'))])
#text_clf.fit(train_data['text'], train_data['label'])

In [ ]:
#docs_test = test_data['text']
#predicted = text_clf.predict(docs_test)
#np.mean(predicted == test_data['label'])

In [ ]:
#test_accs = []
#for i in range(10):
#  model = RobertaClass()
#  model.to(device)
#
#  loss_function = torch.nn.CrossEntropyLoss()
#  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
#
#  for epoch in range(EPOCHS):
#      train(epoch)
#  acc = valid(model, testing_loader)
#  #print("test accuracy = %0.2f%%" % acc)
#  test_accs.append(acc)

# mean_acc = sum(test_accs) / len(test_accs)
# print("MEAN ACCURACY = %0.2f%%" % mean_acc)
# output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/d/d9/Wilhelm_Scream.ogg").play()')
